https://towardsdatascience.com/using-neural-networks-for-a-functional-connectivity-classification-of-fmri-data-ff0999057bc6

In [1]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
from pathlib import Path
import os
import warnings
import glob
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def read_in_matrices(data_dir, file_suffix=None, zscore=False):
    """
    Reads in a set of individual-subject connectivity matrices stored in data_dir,
    
    Returns a dataframe that is subjects x edges (by vectorizing the upper triangle of each FC matrix).
    
    Assumes:
    - each matrix is stored in a separate file beginning with the subject ID, and
    - matrices are symmetric (squareform); i.e., for a parcellation with 268 nodes, matrices should be 268 x 268
    """
    
    all_fc_data = {}
    for subj in glob.glob(data_dir + "*.txt"):
        # try to find this subject's matrix
        file = subj

        
        # read it in and make sure it's symmetric and has reasonable dimensions
        tmp = np.loadtxt(file,  delimiter=',')
        assert tmp.shape[0]==tmp.shape[1]>1, "Matrix seems to have incorrect dimensions: {}".format(tmp.shape)
        subject = next((s for s in file.split("/")[-1].split('.') if file_suffix in s), None) # returns 'AMBAC070' por exemplo

        # take just the upper triangle and store it in a dictionary
        if ~zscore:
            all_fc_data[subject] = tmp[np.triu_indices_from(tmp, k=1)]
        if zscore:
            all_fc_data[subject] = sp.stats.zscore(tmp[np.triu_indices_from(tmp, k=1)])
    
    
    # Convert dictionary into dataframe
    all_fc_data = pd.DataFrame.from_dict(all_fc_data, orient='index')
    
    return all_fc_data

In [3]:
path = "correlation_matrix/"
all_fc_data = read_in_matrices(path, file_suffix='AMBAC')
all_fc_data = all_fc_data.reset_index().drop_duplicates(subset='index', keep='last').set_index('index').sort_index()

In [4]:
all_behav_data = pd.read_csv('resultado_AMBAC.csv', dtype={'Subject': str})
all_behav_data.set_index('fmri_code', inplace=True)


In [5]:
all_behav_data = all_behav_data.loc[all_behav_data.index.intersection(all_fc_data.index)]
all_fc_data = all_fc_data.loc[all_fc_data.index.intersection(all_behav_data.index)]


In [6]:
mergedData = all_behav_data.merge(all_fc_data, left_index=True, right_index=True)

In [7]:
mergedData.columns = mergedData.columns.astype(str)

mergedData.loc[:,'0':'6669']

,0,1,2,3,4,5,6,7,8,9,...,6660,6661,6662,6663,6664,6665,6666,6667,6668,6669
AMBAC002,0.224453,0.026833,-0.115072,-0.107804,-0.119077,0.098518,-0.013074,0.008957,0.054643,0.249948,...,0.529677,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AMBAC005,0.359312,0.246687,0.556772,0.121289,-0.180004,0.382876,0.260317,0.033281,-0.264635,0.423935,...,0.760304,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AMBAC006,0.534393,0.111283,-0.173513,-0.025006,-0.092542,-0.051647,-0.068786,0.103325,-0.325004,0.455359,...,0.586871,0.097291,0.000000,0.042744,0.134844,0.000000,0.021347,0.000000,0.168665,0.000000
AMBAC007,0.493204,0.056914,0.352139,-0.098430,0.106770,-0.038427,0.311217,-0.132460,0.026979,0.228928,...,0.848102,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AMBAC009,0.739659,0.361713,0.579131,-0.021446,0.034835,0.496933,0.539170,0.062104,0.146481,0.656779,...,0.755872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AMBAC091,0.575746,0.456343,0.356181,0.118814,0.435038,0.378848,0.338086,0.312840,0.164663,0.441674,...,0.727718,0.471780,0.328402,0.273524,0.629722,0.456215,0.315639,0.491840,0.372266,0.591903
AMBAC098,0.639946,0.270302,0.368711,0.079681,0.237024,0.015285,0.204195,0.255182,0.249228,0.556441,...,0.605168,0.090958,-0.050603,-0.170203,0.228583,0.181728,-0.118766,0.184700,0.090116,0.139310
AMBAC098,0.639946,0.270302,0.368711,0.079681,0.237024,0.015285,0.204195,0.255182,0.249228,0.556441,...,0.605168,0.090958,-0.050603,-0.170203,0.228583,0.181728,-0.118766,0.184700,0.090116,0.139310
AMBAC099,0.385947,0.264924,0.385979,0.159520,0.493245,0.129837,0.132703,0.016750,0.312444,0.463667,...,0.706635,0.233334,0.066247,0.069966,0.331490,-0.071436,0.005219,0.252227,0.018923,0.499058


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mergedData.loc[:,'0':'6669'], mergedData['Group_cat'], test_size=0.3)

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(32, activation='tanh', kernel_initializer='random_normal', input_shape=connectivity_biomarkers['correlation'].shape[1:]))
#Second Hidden Layer
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#Third Hidden Layer
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#Compiling the model
classifier.compile(optimizer = Adam(lr =.0001),loss='binary_crossentropy', metrics =['accuracy'])
#Fitting the model
classifier.fit(np.array(X_train),np.array(y_train), batch_size=32, epochs=100)

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
eval_model=classifier.evaluate(np.array(X_train), np.array(y_train))
eval_model

In [ ]:
y_pred=classifier.predict(X_test,batch_size=32)
y_pred =(y_pred>0.5)
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
cr = classification_report(y_test, y_pred)
print(cr)
